In [1]:
import requests
import json

# Define the diseases to search for
diseases = ["Irritable Bowel Syndrome", "Celiac Disease", "Crohn's Disease"]

# Define the base URL for the ClinicalTrials.gov API
base_url = "https://clinicaltrials.gov/api/query/full_studies"

# Function to get clinical trials data for a specific disease
def get_clinical_trials(disease):
    params = {
        "expr": disease,
        "min_rnk": 1,
        "max_rnk": 50,  # Adjust the range based on your needs
        "fmt": "json"
    }
    response = requests.get(base_url, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        return None


/Users/anantvishwakarma/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [4]:
# Function to extract drug information from clinical trials data
def extract_drugs(data):
    drugs = []
    if 'FullStudiesResponse' in data and 'FullStudies' in data['FullStudiesResponse']:
        for study in data['FullStudiesResponse']['FullStudies']:
            arms_interventions = study['Study']['ProtocolSection'].get('ArmsInterventionsModule', {})
            intervention_list = arms_interventions.get('InterventionList', {})
            interventions = intervention_list.get('Intervention', [])
            for intervention in interventions:
                if intervention['InterventionType'] == 'Drug':
                    drugs.append(intervention['InterventionName'])
    return list(set(drugs))


In [5]:
# Dictionary to hold the results
results = {}

# Loop through each disease and get the clinical trials data
for disease in diseases:
    trials_data = get_clinical_trials(disease)
    if trials_data:
        drugs = extract_drugs(trials_data)
        results[disease] = drugs

# Save the results to a JSON file
output_file = "clinical_trials_drugs.json"
with open(output_file, "w") as file:
    json.dump(results, file, indent=4)

print(f"Results saved to {output_file}")

Results saved to clinical_trials_drugs.json


In [12]:
import requests
import json
from collections import Counter

# Define the diseases to search for
diseases = ["Irritable Bowel Syndrome", "Celiac Disease", "Crohn's Disease"]

# Define the base URL for the ClinicalTrials.gov API
base_url = "https://clinicaltrials.gov/api/query/full_studies"

# Function to get clinical trials data for a specific disease
def get_clinical_trials(disease, max_rank=100):
    all_studies = []
    current_rank = 1

    while current_rank <= max_rank:
        params = {
            "expr": disease,
            "min_rnk": current_rank,
            "max_rnk": min(current_rank + 99, max_rank),  # Fetch in chunks of 100
            "fmt": "json"
        }
        response = requests.get(base_url, params=params)
        if response.status_code == 200:
            data = response.json()
            if 'FullStudiesResponse' in data and 'FullStudies' in data['FullStudiesResponse']:
                all_studies.extend(data['FullStudiesResponse']['FullStudies'])
            else:
                break
        else:
            print(f"Failed to fetch data for {disease} starting from rank {current_rank}")
            break
        current_rank += 100

    return all_studies

# Function to extract drug information from clinical trials data
def extract_drugs(studies):
    drugs = []
    for study in studies:
        arms_interventions = study['Study']['ProtocolSection'].get('ArmsInterventionsModule', {})
        intervention_list = arms_interventions.get('InterventionList', {})
        interventions = intervention_list.get('Intervention', [])
        for intervention in interventions:
            if intervention['InterventionType'] == 'Drug':
                drugs.append(intervention['InterventionName'])
    return drugs

# Dictionary to hold the results
results = {}

# Loop through each disease and get the clinical trials data
for disease in diseases:
    print(f"Fetching clinical trials for {disease}...")
    studies = get_clinical_trials(disease, max_rank=500)  # Increase max_rank as needed
    if studies:
        drugs = extract_drugs(studies)
        results[disease] = Counter(drugs)
    else:
        print(f"No data found for {disease}")

# Save the results to a JSON file
output_file = "clinical_trials_drugs_counts.json"
with open(output_file, "w") as file:
    json.dump(results, file, indent=4)

# Display the results and get top 5 drugs for each disease
top_5_drugs_per_disease = {}

for disease, drug_counter in results.items():
    print(f"\n{disease}:")
    top_5_drugs = drug_counter.most_common(5)
    top_5_drugs_per_disease[disease] = top_5_drugs
    for drug, count in top_5_drugs:
        print(f"{drug}: {count}")

# Save the top 5 drugs for each disease to a JSON file
output_top_5_file = "top_5_drugs_per_disease.json"
with open(output_top_5_file, "w") as file:
    json.dump(top_5_drugs_per_disease, file, indent=4)

print(f"Top 5 drugs for each disease saved to {output_top_5_file}")



Fetching clinical trials for Irritable Bowel Syndrome...
Fetching clinical trials for Celiac Disease...
Fetching clinical trials for Crohn's Disease...

Irritable Bowel Syndrome:
Placebo: 86
placebo: 11
Linaclotide: 8
Rifaximin: 8
Ibodutant: 6

Celiac Disease:
Placebo: 18
larazotide acetate: 5
placebo: 4
Simvastatin: 3
Latiglutenase: 3

Crohn's Disease:
Placebo: 72
Ustekinumab: 18
Infliximab: 11
Adalimumab: 8
CP-690,550: 7
Top 5 drugs for each disease saved to top_5_drugs_per_disease.json
